In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/train.csv")
d = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/test.csv")
df

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
for col in df.columns:
    print(col)
    print(df[col].nunique())

In [ ]:
def date(x):
    return x.split()[0]
def time(x):
    return x.split()[1]
df['Date'] = df['time'].apply(date)
df['Time'] = df['time'].apply(time)
df.pop('time')
df

In [ ]:
pd.to_datetime(df['Date']).dt.year.unique()


In [ ]:
def month(x):
    return int(x.split('-')[1])
df['Month'] = df['Date'].apply(month)
df.pop('Date')
df

In [ ]:
def hour(x):
    return int(x.split(':')[0])
df['Hour'] = df['Time'].apply(hour)
df.pop('Time')
df

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
df = pd.get_dummies(data = df,  columns=["direction"], drop_first = False)
df

In [ ]:
X=df[['x', 'y', 'Month', 'Hour', 'direction_EB',
       'direction_NB', 'direction_NE', 'direction_NW', 'direction_SB',
       'direction_SE', 'direction_SW', 'direction_WB']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,df.congestion,test_size=0.2,random_state=0)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc_x = StandardScaler()
# X_train = sc_x.fit_transform(X_train) 
# X_test = sc_x.transform(X_test)

In [ ]:
from sklearn import preprocessing
sc_x = preprocessing.RobustScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

In [ ]:
import xgboost as xg
# from sklearn.metrics import mean_squared_error as MSE
# xgb_r = xg.XGBRegressor(n_estimators=999)
  
# # Fitting the model
# xgb_r.fit(X_train, y_train)
  
# # Predict the model
# pred = xgb_r.predict(X_test)
  
# # RMSE Computation
# rmse = np.sqrt(MSE(y_test, pred))
# print("RMSE : % f" %(rmse))

In [ ]:
import matplotlib.pyplot as plt

from xgboost import XGBRegressor 
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor(n_estimators=999)

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,  
                             #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [ ]:
# hyperParameterTuning(X_train, y_train)

In [ ]:
# xgb_r = XGBRegressor(
#         objective = 'reg:squarederror',
#         colsample_bytree = 0.5,
#         learning_rate = 0.05,
#         max_depth = 6,
#         min_child_weight = 1,
#         n_estimators = 1000,
#         subsample = 0.7)

# %time xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_val, y_val)], verbose=False)

# y_pred_xgb = xgb_model.predict(X_val)
# mae_xgb = mean_absolute_error(y_val, y_pred_xgb)

# print("MAE: ", mae_xgb)

In [ ]:
xgb_r = xg.XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.7,
        learning_rate = 0.1,
        max_depth = 8,
        min_child_weight = 3,
        n_estimators = 1000,
        subsample = 0.5) 

# 'learning_rate': [0.01, 0.1],
#         'max_depth': [3, 5, 7, 10],
#         'min_child_weight': [1, 3, 5],
#         'subsample': [0.5, 0.7],
#         'colsample_bytree': [0.5, 0.7],
#         'n_estimators' : [100, 200, 500],
#         'objective': ['reg:squarederror']

xgb_r.fit(X_train, y_train)

y_pred_xgb = xgb_r.predict(X_test)
rmse = np.sqrt(MSE(y_test, y_pred_xgb))
print("RMSE: ", rmse)

# mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
# print("MAE: ", mae_xgb)

In [ ]:
d['Month'] = d['Date'].apply(month)
d.pop('Date')

In [ ]:
d['Hour'] = d['Time'].apply(hour)
d.pop('Time')

In [ ]:
d = pd.get_dummies(data = d,  columns=["direction"], drop_first = False)
d

In [ ]:
X=d[['x', 'y', 'Month', 'Hour', 'direction_EB',
       'direction_NB', 'direction_NE', 'direction_NW', 'direction_SB',
       'direction_SE', 'direction_SW', 'direction_WB']]

In [ ]:
X_test = sc_x.transform(X)

In [ ]:
x_test_pred = xgb_r.predict(X_test)

In [ ]:
data = {'row_id':list(d['row_id']),
        'congestion':list(x_test_pred)}
 
# Create DataFrame
answer = pd.DataFrame(data)
answer

In [ ]:
answer.to_csv('submission.csv', index = False)
answer.head()